In [4]:
import numpy as np
import random
from hashlib import sha1

## 1. Generierung der Parameter
Generiere
 * Primzahl $q$ der Länge $N = 160$
 * Primzahl $p = qk + 1, k \in \mathbb{N}$ der Länge $L = 1024$
 * Wähle zufälliges $1 < h < p - 1$, sodass $g = h^{\frac{p-1}{q}} \mod p \neq 1$

In [5]:
# aus Task 6
def quad_and_mult(x, m, n):
    """Berechnet effizient x^m mod n."""
    y = 1
    while m != 0:
        if m & 0x1 != 0:
            # falls bit=1: Multipliziere y mit x
            y = (y * x) % n
        # für jedes Bit wird x quadriert
        x = (x * x) % n
        # schiebe zum nächsten Bit
        m >>= 1
    return y

In [6]:
# aus Task 7 (RSA Keygen)
def miller_rabin_test(n):
    """Ist n prim? Gibt mit p < 1/4 True zurück, falls n zusammengesetzt ist (False Positive)
    und sicher True, falls n eine Primzahl ist."""
    assert n > 2
    # Bestimme ungerades m mit n - 1 = 2^k * m
    m = n - 1
    k = 0
    while m & 0b1 == 0:
        m >>= 1
        k += 1
    
    # Wähle zufälliges 2 <= a < n
    a = random.randrange(2, n)

    # b = a^m mod n
    b = quad_and_mult(a, m, n)

    if b == 1:
        return True
    for i in range(k):
        if b == n - 1:
            return True
        else:
            b = (b * b) % n
    return False

# aus Task 7 (RSA Keygen)
def is_prime(n):
    """Gibt True zurück, wenn mit sehr hoher Wahrscheinlichkeit (0.999999) prim ist. """
    for i in range(10):
        if miller_rabin_test(n) == False:
            return False
    # mit p > 1 - (1/4)**10 = 0.999999... ist n prim
    return True

In [7]:
# Abgewandelt von look_for_prime (RSA Keygen)
def look_for_prime(z):
    # Sucht eine Primzahl q > 30 * z
    for i in [1, 7, 11, 13, 17, 19, 23, 29]:
        q = 30 * z + i
        if is_prime(q):
            return q
    # Keine Primzahl gefunden, erhöhe z
    return look_for_prime(z + 1)

def find_qp(L, N, k_tries = int(1e6)):
    # Las-Vegas-Algorithmus zur Suche zweier Primzahlen q (Bitlänge N) und p = qk + 1 (Bitlänge L), k=2,3,...
    z = (((0b1 << (N - 1)) ^ random.getrandbits(N - 1)) + 30) // 30
    q = look_for_prime(z)
    if q.bit_length() != N:
        print(f"bit length is {q.bit_length()} (expected: {N})")
        # neuer Versuch für neues q
        return find_qp(L, N)
    # Suche passendes q
    # Grobes Intervall für k (Bitlänge wird in Schleife verifiziert)
    min_k = 2 ** (L - N)
    max_k = 2 ** (L - N + 1) - 1
    for t in range(k_tries):
        k = random.randint(min_k, max_k)
        p = q * k + 1
        if p.bit_length() == L and is_prime(p):
            return q, p
    # alle k's probiert, neuer Versuch für p
    print(f"{k_tries} exceeded, generating new q")
    return find_qp(L, N)

In [8]:
# Test von find_qp
q, p = find_qp(1024, 160)
print(q.bit_length())
print(p.bit_length())
assert q.bit_length() == 160
assert p.bit_length() == 1024
# Muss gelten, da p = qk + 1
assert (p - 1) % q == 0

160
1024


In [9]:
def find_params(L = 1024, N = 160):
    # Generiert Parameter q, p und Pseudo-Generator g mit Ordnung q
    q, p = find_qp(L, N)
    g = 1
    # Muss gelten, da p = qk + 1
    assert (p - 1) % q == 0
    # Finde g = h^((p-1)/q) mod p != 1
    while g == 1:
        h = random.randrange(2, p - 1)
        g = quad_and_mult(h, (p - 1) // q, p)
    return q, p, g

In [10]:
# Test von find_params
q, p, g = find_params()
assert q.bit_length() == 160
assert p.bit_length() == 1024
assert (p - 1) % q == 0
print(f"g = {g}")

g = 16725027302444251933037352332337165132631710771364493089116573492216960287116937882041382367161404505384407614166020010504080888197348125035494497483695061902450233017743969996548070549086304922493401662729111571356191101123237883251904191951639247980026548456874390453212411845057560070753881749266508095828


## 2. Schlüsselgenerierung
 * geheimer Schlüssel $x$
 * öffentlicher Schlüssel $y$

In [11]:
def gen_keys(q, p, g):
    # Generiert geheimen Schlüssel x, 1 < x < q und öffentlichen Schlüssel y = g^x mod p
    x = random.randrange(2, q)
    y = quad_and_mult(g, x, p)
    return x, y

In [12]:
x, y = gen_keys(q, p, g)
assert 1 < x < q
print(f"x = {x}")
print(f"y = {y}")

x = 979868034643036533728838303592760424900204073027
y = 88271768894623792573087514634063057080912267195975871575406880686305975227918437952276766923860892588008565012868232672399771672011199421555624317951094087758406718796550313250700257274709404209444350715502951023290277121514056817073021070168641887983868791398202828537666391200636730741597167993687511683749


## 3. Signieren

In [13]:
# kopiert aus Übung 8
def hash_func(input: bytes) -> bytes:
    return sha1(input).digest()

# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [14]:
# kopiert aus Übung 6
def extended_euclidian(a, b):
    """ Berechnet ggT(a,b) = sa + tb = r und gibt r, s, t zurück. """
    r = [a, b]
    s = [1, 0]
    t = [0, 1]
    while r[-1] != 0:
        q = r[-2] // r[-1]
        r.append(r[-2] - q * r[-1])
        s.append(s[-2] - q * s[-1])
        t.append(t[-2] - q * t[-1])
    return r[-2], s[-2], t[-2]

In [15]:
def sign(m: bytes, q: int, p: int, g: int, x: int, hash_func):
    # Signieren nur Hashwert der Nachricht m
    hash_m = int_from_bytes(hash_func(m))
    while True:
        # Benötigt für jede Nachricht unabhängiges 1 < j < q
        j = random.randrange(2, q)
        # Berechne r = (g^j mod p) mod q
        r = quad_and_mult(g, j, p) % q
        # Falls r = 0, neues j wählen
        if r == 0:
            continue
        # Berechne Inverses von j mod q
        ggT, j_inv, _ = extended_euclidian(j, q)
        j_inv %= q
        assert ggT == 1
        assert 0 < j_inv < q
        # Berechne s = j^(-1) * (hash(m) + rx) mod q
        s = (j_inv * (hash_m + r * x)) % q
        # Falls s = 0, ebenfalls neues j wählen
        if s == 0:
            continue
        # Tupel (r,s) ist Signatur
        return r, s

# Test
message = b"Hello crypto world"
r, s = sign(message, q, p, g, x, hash_func)
assert 0 < r < q and 0 < s < q
print(f"r = {r}, s = {s}")

r = 330717866946389438125356413717477363207558021733, s = 17352639326113800251095180980800073151012998055


In [16]:
def verify(m: bytes, r: int, s: int, q: int, p: int, g: int, y: int, hash_func) -> bool:
    if 0 < r < q and 0 < s < q:
        hash_m = int_from_bytes(hash_func(m))
        # Berechne w = s^(-1) mod q
        ggT, w, _ = extended_euclidian(s, q)
        w %= q
        assert ggT == 1
        assert 0 < w < q
        # Berechne u1 = hash(m) * w mod q
        u1 = (hash_m * w) % q
        # Berechne u2 = rw mod q
        u2 = (r * w) % q
        # Berechne v = (g^u1 y^u2 mod p) mod q
        v = ((quad_and_mult(g, u1, p) * quad_and_mult(y, u2, p)) % p) % q
        print(f"v = {v}, r = {r}")
        return v == r
    else:
        print("wrong ranges for r, s")
        return False

# Test
assert verify(message, r, s, q, p, g, y, hash_func)

v = 330717866946389438125356413717477363207558021733, r = 330717866946389438125356413717477363207558021733


In [17]:
-2 % 5

3

In [21]:
is_prime(1428505320555825478976864933123470205494890763411)

False